In [8]:
# First cell: File upload with sanitization
from google.colab import files
import os

print("Step 1: Upload your video file")
uploaded = files.upload()

# Sanitize filename
input_filename = next(iter(uploaded.keys()))
clean_filename = input_filename.replace(" ", "_").replace("(", "").replace(")", "")
os.rename(input_filename, clean_filename)

input_video = f'/content/{clean_filename}'
print(f"\nUploaded file: {clean_filename}")

Step 1: Upload your video file


Saving slow_office-handshake-stock-footage_RogDvWOy.mp4 to slow_office-handshake-stock-footage_RogDvWOy.mp4

Uploaded file: slow_office-handshake-stock-footage_RogDvWOy.mp4


In [9]:
# Second cell: Full implementation with intersection detection
!pip install opencv-python mediapipe > /dev/null
!apt install ffmpeg > /dev/null

import cv2
import mediapipe as mp
import numpy as np
import os

# Generate output filename
output_video = f'/content/processed_{os.path.basename(input_video)}'

# Remove existing output file
if os.path.exists(output_video):
    os.remove(output_video)

# Convert video to safe format
converted_input = "/content/converted_input.mp4"
!ffmpeg -i "{input_video}" -vcodec libx264 -an -crf 23 "{converted_input}" -y

# Initialize MediaPipe Hands with better detection parameters
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=5,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    model_complexity=0)

# Video processing setup
cap = cv2.VideoCapture(converted_input)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = cap.get(5)

# Initialize VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

# Visualization settings
COLORS = [(0, 255, 0), (0, 0, 255)]  # Green and Red for different hands
INTERSECTION_COLOR = (255, 0, 0)      # Blue for intersection
IOU_THRESHOLD = 0.05                  # Intersection threshold

def calculate_iou(box1, box2):
    """Calculate Intersection over Union for two bounding boxes"""
    # Get coordinates
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # Calculate area of intersection
    intersection = max(0, x2 - x1) * max(0, y2 - y1)

    # Calculate areas of individual boxes
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Calculate union area
    union = area1 + area2 - intersection

    return intersection / union if union != 0 else 0

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    hand_boxes = []

    if results.multi_hand_landmarks:
        for hand_idx, hand in enumerate(results.multi_hand_landmarks):
            # Draw hand landmarks
            mp_drawing.draw_landmarks(
                frame, hand, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=COLORS[hand_idx % 2], thickness=2),
                mp_drawing.DrawingSpec(color=COLORS[hand_idx % 2], thickness=2))

            # Get bounding box coordinates
            x_coords = [lm.x * frame_width for lm in hand.landmark]
            y_coords = [lm.y * frame_height for lm in hand.landmark]
            min_x, max_x = int(min(x_coords)), int(max(x_coords))
            min_y, max_y = int(min(y_coords)), int(max(y_coords))
            hand_boxes.append((min_x, min_y, max_x, max_y))

            # Draw bounding box
            cv2.rectangle(frame,
                         (min_x, min_y), (max_x, max_y),
                         COLORS[hand_idx % 2], 2)

    # Detect intersections between hands
    if len(hand_boxes) >= 2:
        iou = calculate_iou(hand_boxes[0], hand_boxes[1])

        if iou > IOU_THRESHOLD:
            # Draw intersection rectangle
            ix1 = max(hand_boxes[0][0], hand_boxes[1][0])
            iy1 = max(hand_boxes[0][1], hand_boxes[1][1])
            ix2 = min(hand_boxes[0][2], hand_boxes[1][2])
            iy2 = min(hand_boxes[0][3], hand_boxes[1][3])
            cv2.rectangle(frame, (ix1, iy1), (ix2, iy2), INTERSECTION_COLOR, 3)

            # Add intersection text
            cv2.putText(frame, "HANDS INTERSECTING!", (50, 80),
                       cv2.FONT_HERSHEY_SIMPLEX, 1.2, INTERSECTION_COLOR, 3,
                       cv2.LINE_AA)

            # Flash frame border
            if frame_count % 10 < 5:
                cv2.rectangle(frame, (0,0), (frame_width-1, frame_height-1),
                             INTERSECTION_COLOR, 50)

    # Write frame to output
    out.write(frame)
    frame_count += 1

cap.release()
out.release()

# Final verification
print(f"\nProcessed {frame_count} frames")
print(f"Output file size: {os.path.getsize(output_video)/1024/1024:.2f}MB")



ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --en